<a href="https://colab.research.google.com/github/EltonWolf/GoogleEarthPythonBootcampAPI-/blob/main/Landsat_Visualisation_Mozambique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee

In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=P5dOrwxBTRjneki2_YMfkZ6qLH7uDEzHe76kj0xfZUY&tc=FpBePd_WvhInpvgICJI5-LiERW-JIi4VXX7gCZj935c&cc=mAInHZM7UVuWN5b_u9eF1ttz9F3cPgcXA7ayaVI0Z8U

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXmn8waiLg73XicaPhC28NSFy5H7jdZPrfFUbz2p-cwlHCbbCMCcqSA

Successfully saved authorization token.


In [33]:
#Load data for countries
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
Mozambique = countries.filter(ee.Filter.eq("country_na","Mozambique"))

#Loading Landsat
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1").filterDate("2017-01-01", "2018-01-01").filterBounds(Mozambique)
composite = ee.Algorithms.Landsat.simpleComposite(**{
    'collection':landsat,
    'asFloat':True
})

rgbVis = {'bands': ['B4','B3','B2'], 'min': 0, 'max': 0.3}
nirVis = {'bands': ['B5','B4','B3'], 'min': 0, 'max': [0.5, 0.3, 0.3]}

In [34]:
import folium

In [38]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[-18.6657, 35.5296], zoom_start = 4.5)

# Add the layer to the map object.
my_map.add_ee_layer(composite, rgbVis, "RGB")
my_map.add_ee_layer(composite, nirVis, "False Color")

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)